# Chuẩn bị và cấu hình source

## Kiểm tra GPU

In [ ]:
!nvidia-smi

## Clone source code từ github và build



Clone source darknet của AlexeyAB từ github về. <br>
Đây là source code tiêu chuẩn để bắt đầu với YOLOv3.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet --depth=1
%cd darknet

Cần tinh chỉnh các tham số trong file config để build darknet dành cho GPU.

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!sed -i "s/\$(CPP)/\$(CPP) -w/" Makefile
!sed -i "s/\$(CC)/\$(CC) -w/" Makefile
!sed -i "s/\$(NVCC)/\$(NVCC) -w/" Makefile
!make --silent

## Tùy chỉnh file config cho YOLOv3

Tạo một bản sao của file config gốc

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg



Trong file cfg/yolov3_training.cfg cần quan tâm các thông số:

*   batch = 64
<font color='#339B33'>
    # Batch size = batch/subdivision = 4
</font>
*   subdivisions = 16
<font color='#339B33'>
    # Chu kì để xuất loss
</font>
*   max_batches = 4000
<font color='#339B33'>
    # max_batches nên để bằng (số class) * 2000, tuy nhiên để demo thì chỉ cần để số batches nhỏ
</font>
*   steps = 3200, 3600 
<font color='#339B33'>
    # steps = 0.8 * max_batches, 0.9 * max_batches
</font>
*   width = 416, height = 416
<font color='#339B33'>
    # Kích thước đầu vào của mạng, là một số chia hết cho 32.
</font>
*   Ngoài ra còn cần sửa lại số classes và số filters theo công thức filters=(classes+5) * 3

Trong folder data:
*   Tạo obj.names trong đó mỗi dòng chứa tên của 1 class
*   Tạo obj.data Có nội dung như sau:

classes = 6 
<font color='#339B33'>
    \# Số lượng classes trong bài toán detection.
</font> <br>
train  = /content/train.txt 
<font color='#339B33'>
    \# Đường dẫn đến tệp chứa danh sách các ảnh train.
</font> <br>
valid  = /content/valid.txt 
<font color='#339B33'>
    \# Đường dẫn đến tệp chứa danh sách các ảnh validation.
</font> <br>
names = data/obj.names 
<font color='#339B33'>
    \# Đường dẫn đến tệp chứa danh sách tên của các classes.
</font> <br>
backup = backup/
<font color='#339B33'>
    \# Đường dẫn chứa các bộ trọng số sau khi train.
</font> <br>

In [ ]:
!sed -i -zE \
-e 's/(\nbatch)[^\n]+\n/\1=64\n/' \
-e 's/(\nsubdivisions)[^\n]+\n/\1=16\n/' \
-e 's/(\nmax_batches)[^\n]+\n/\1=4000\n/' \
-e 's/(\nsteps)[^\n]+\n/\1=3200,3600\n/' \
-e 's/(\nwidth)[^\n]+\n/\1=416\n/' \
-e 's/(\nheight)[^\n]+\n/\1=416\n/' \
-e 's/(\nfilters)[^\n]+(\n[^\n]+\n\n\n[^\n]+\n[^\n]+\n[^\n]+)(\nclasses)[^\n]+/\1=33\2\3=6/g' \
cfg/yolov3_training.cfg

In [ ]:
!printf "UIT\nHSV\nCS\nCE\nSE\nISE" > data/obj.names
!printf "classes=6\ntrain=/content/train.txt\nvalid=/content/valid.txt\nnames=data/obj.names\nbackup=backup/" > data/obj.data

# Chuẩn bị dữ liệu



## Tải và chuyển đổi định dạng dữ liệu

Sử dụng lệnh gdown để tải xuống dữ liệu mà Ban Tổ Chức cuộc thi cung cấp. Dữ liệu sẽ được tải về và giải nén vào thư mục /content/train.

In [ ]:
%cd /content
!gdown --id 1onsbmBLR279EG1Sgx7YfQ8LsKgBoKDQn -O train.zip
!unzip -j -qq train.zip -d train

/content
Downloading...
From: https://drive.google.com/uc?id=1onsbmBLR279EG1Sgx7YfQ8LsKgBoKDQn
To: /content/train.zip
1.89GB [00:14, 130MB/s] 


Chúng ta cần chuyển định dạng dữ liệu của BTC về đúng định dạng yêu cầu của phương pháp chúng ta chọn, trong trường họp này là YOLOv3.

In [ ]:
import os
from PIL import Image

DATASET_PATH = '/content/train/'
# Đọc tất cả các tệp tin trong thư mục dữ liệu train
images_list = os.listdir(DATASET_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm file hình)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
# Thêm đường dẫn /content/train/ vào trước tên tệp tin
images_list = [os.path.join(DATASET_PATH, filename) for filename in images_list]

for filename in images_list:
    
    # Đọc ảnh để lấy kích thước của ảnh
    img = Image.open(filename)
    width, height = img.size

    # Tìm tệp tin chứa nhãn dữ liệu tương ứng với ảnh
    annotation_file = os.path.splitext(filename)[0] + '.txt'

    # Đọc tệp tin chứa nhãn dữ liệu và tách thành từng dòng
    with open(annotation_file, 'r') as fi:
        lines = fi.read().splitlines()

    # Mở lại tệp tin nhãn để ghi đè
    with open(annotation_file, 'w') as fo:
        # Duyệt qua từng dòng trong tệp tin
        for line in lines:
            # Tách nội dung của từng dòng theo định dạng của BTC
            line = line.strip().split()
            # Chuyển đổi nội dung từ string sang int
            line = [int(x) for x in line]

            # Tiền xử lý: nếu các tọa độ ko nằm trong ảnh thì xóa logo đi (đây là nhiễu)
            if line[1] <= 0 or  line[3] > width:
                continue
            if line[2] <= 0 or  line[4] > height:
                continue

            # Chuyển tọa độ xmin, ymin, xmax, ymax về tọa độ x_center, y_center, w, h
            # Lưu ý: YOLOv3 yêu cầu chuẩn hóa các tọa độ về khoảng [0; 1].
            line = [
                line[0],
                (line[3] + line[1])/2/width,    # x_center = (xmax + xmin)/2
                (line[4] + line[2])/2/height,   # y_center = (ymax + ymin)/2
                (line[3] - line[1])/width,      # w = (xmax - xmin)
                (line[4] - line[2])/height,     # h = (ymax - ymin)
            ]

            # Tiền xử lý: nếu w hoặc h của logo bằng 0 thì bỏ qua logo này
            if line[3] <= 0 or line[4] <= 0:
                continue

            # Tiền xử lý: nếu tỉ lệ giữa w và h lớn hơn 10 thì bỏ qua logo này
            # (logo thường là hình vuông hoặc hình chữ nhật với tỉ lệ cân đối)
            if line[3] / line[4] > 10 or line[4] / line[3] > 10:
                continue

            # Chuyển các tọa độ về kiểu string
            line = [str(x) for x in line]
            # Đưa về định dạng của YOLOv3
            line = ' '.join(line) + '\n'
            # Ghi vào tệp tin output
            fo.write(line)

## Chia tập dữ liệu train và validation

Tạo file train.txt và valid.txt chứa danh sách các file ảnh được dùng để train và validation cho YOLOv3. <br>
<i>(Tập train và tập valid được chia theo tỉ lệ 8:2) </i>

In [ ]:
import os
import random
random.seed(42)

DATASET_PATH = '/content/train/'
# Đọc tất cả các tệp tin trong thư mục dữ liệu train
images_list = os.listdir(DATASET_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm những file ảnh)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
# Thêm đường dẫn /content/train/ vào trước tên tệp tin
images_list = [os.path.join(DATASET_PATH, filename) for filename in images_list]

# Chọn 80% ngẫu nhiên ra làm tập dữ liệu train
train_list = random.sample(images_list, len(images_list)*8//10)
# Phần còn lại chính là tập dữ liệu validation
valid_list = list(set(images_list) - set(train_list))

# Sắp xếp lại hai tập dữ liệu cho đẹp :)
train_list.sort()
valid_list.sort()

# Cuối cùng chỉ cần xuất 2 tập dữ liệu ra 2 danh sách tương ứng
f = open("train.txt", "w")
f.write("\n".join(train_list))
f.close()

f = open("valid.txt", "w")
f.write("\n".join(valid_list))
f.close()

# Bắt đầu train!

Tải xuống bộ trọng số (weights) có sẵn của darknet53 và tiến hành train tiếp YOLOv3 trên bộ trọng số đó.

In [ ]:
%cd darknet
!wget https://pjreddie.com/media/files/darknet53.conv.74

Bắt đầu train thôi nào ^_^ !

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show -map 2>/dev/null

# Predicting

## Tải tập dữ liệu warm-up

Tương tự với tập dữ liệu train, ở đây chúng ta cũng sử dụng lệnh gdown để tải tập dữ liệu warmup của Ban Tổ Chức về.

In [ ]:
%cd /content
!gdown --id 1Ng8UWSg2XNMWS49qhchTz24wSfE80cfA -O warmup.zip
!unzip -j -qq warmup.zip -d warmup

Ở giai đoạn này cũng phải tạo ra một danh sách chứa các ảnh cần predict.

In [ ]:
import os
import random
random.seed(42)

WARMUP_PATH = '/content/warmup/'
# Đọc tất cả các tệp tin trong thư mục dữ liệu warmup
images_list = os.listdir(WARMUP_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm những file ảnh)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
# Thêm đường dẫn /content/warmup/ vào trước tên tệp tin
images_list = [os.path.join(WARMUP_PATH, filename) for filename in images_list]

# Sắp xếp lại danh sách
images_list.sort()

# Cuối cùng là xuất danh sách này ra tệp tin txt
f = open("warmup.txt", "w")
f.write("\n".join(images_list))
f.close()

## Bắt đầu predict!

In [ ]:
%cd /content/darknet

In [ ]:
!./darknet detector test data/obj.data cfg/yolov3_training.cfg backup/yolov3_training_best.weights -thresh 0.25 -dont_show -save_labels < /content/warmup.txt

# Nộp bài

In [ ]:
%cd /content

In [ ]:
import os
from PIL import Image

WARMUP_PATH = '/content/warmup/'
# Đọc tất cả các tệp tin trong thư mục dữ liệu train
images_list = os.listdir(WARMUP_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm file hình)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']

submission_file = 'submission.txt'

with open(submission_file, 'w') as fo:
    for filename in images_list:
        
        # Thêm đường dẫn /content/warmup/ vào trước tên tệp tin
        file_path = os.path.join(WARMUP_PATH, filename)

        # Đọc ảnh để lấy kích thước của ảnh
        img = Image.open(file_path)
        width, height = img.size

        # Tìm tệp tin chứa nhãn dữ liệu mô hình dự đoán tương ứng với ảnh
        annotation_file = os.path.splitext(file_path)[0] + '.txt'

        # Đọc tệp tin chứa nhãn dữ liệu mô hình dự đoán và tách thành từng dòng
        with open(annotation_file, 'r') as fi:
            lines = fi.read().splitlines()

        # Duyệt qua từng dòng trong tệp tin
        for line in lines:
            # Tách nội dung của từng dòng ra thành các tọa độ
            line = line.strip().split()
            # Chuyển đổi nội dung từ string sang float
            line = [float(x) for x in line]
            # Chuyển định dạng class x_center y_center w h về định dạng submission.
            line = [
                filename,
                int(line[0]),
                int((line[1] - line[3]/2) * width),    # x_min = x_center - w/2
                int((line[2] - line[4]/2) * height),   # y_min = y_center - h/2
                int((line[1] + line[3]/2) * width),    # x_max = x_center + w/2
                int((line[2] + line[4]/2) * height),   # y_max = y_center + h/2
            ]
            # Chuyển các tọa độ về kiểu string
            line = [str(x) for x in line]
            # Đưa về định dạng submission của BTC (cách nhau bởi dấu phẩy)
            line = ', '.join(line) + '\n'
            # Ghi vào tệp tin output
            fo.write(line)

Sau khi các công đoạn hậu xử lý và tạo tệp tin submission.txt hoàn tất, chúng ta có thể tiến hành nộp bài lên hệ thống cuộc thi bằng đoạn mã ngắn dưới đây:

In [ ]:
import math, requests

def submit_prediction(df, comment='', **kwargs):
    # Token các bạn lấy từ trang submission của cuộc thi và dán vào dòng bên dưới
    TOKEN='' 
    URL='http://submission.mmlab.uit.edu.vn/api/submissions'
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': open(df, 'rb')},data={'comment':comment})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)
        
submit_prediction('submission.txt', comment='Submitted from Baseline Colab Notebook.')